In [2]:
import pandas as pd

n_campi = 'CAFI'

levantamento = pd.read_excel('consolidacao/levantamento.xlsx')
suap = pd.read_excel('consolidacao/relatorio_CAFI.xlsx')

# removendo itens nulos e deixando apenas os retificados
retificacao = levantamento[levantamento['Retificação'].str.contains('Retificação').fillna(False)]
# lista de itens vencidos
vencido = levantamento[levantamento['Material Vencido - MV'].str.contains('x').fillna(False)]
# lista de itens não identificados
nao_identificado = levantamento[levantamento['Sem identificação - SI'].str.contains('x').fillna(False)]


# planilha contendo apenas itens válidos
levantamento_ret = levantamento.drop_duplicates(subset=['Código', 'Retificação'], keep='last')
levantamento_ret = levantamento_ret.rename(columns={'Código': 'codigo'})
# elemento despesa do levantamento
ed_lev = levantamento_ret['codigo'].str[3:5]


# adicionando ed ao data frame geral
# levantamento_ret['ED'] = ed_lev

# lista de servidores
servidores = levantamento['Endereço de e-mail'].drop_duplicates(keep='last')
# reindexando uutilizando list comprehension
servidores.index = [x for x in range(0, len(servidores))]

levantamento_cross = levantamento_ret.drop(columns=['Material de pouca movimentação - PM', 'Material Vencido - MV',
                                                 'Material próximo ao vencimento - PV', 'Material danificado ou avariado - MD',
                                                 'Material Obsoleto - MO', 'Sem identificação - SI', 'Valor unitário',
                                                 'Campus', 'Retificação', 'Carimbo de data/hora'])
levantamento_cross = levantamento_cross.rename(columns={'Endereço de e-mail': 'servidor', 'Nome': 'descricao_lev',
                                                        'Quantidade': 'qtd_lev'})
levantamento_cross = levantamento_cross[['codigo', 'descricao_lev', 'qtd_lev', 'servidor']]

suap_cross = suap.rename(columns={'descricao': 'descricao_suap', 'qtd': 'qtd_suap'})
suap_cross = suap_cross.drop(columns=['vlr_total', 'vlr_unt'])
display(suap_cross)
display(levantamento_cross)


,codigo,descricao_suap,qtd_suap,ed
0,30.14.117,"APITO, MATERIAL PLÁSTICO, APLICAÇÃO ESPORTE, T...",16,14 - MATERIAL EDUCATIVO E ESPORTIVO
1,30.14.120,BOLA TENIS DE MESA,17,14 - MATERIAL EDUCATIVO E ESPORTIVO
2,30.14.121,REDE ESPORTE BASQUETE,8,14 - MATERIAL EDUCATIVO E ESPORTIVO
3,30.14.122,REDE ESPORTE FUTSAL,10,14 - MATERIAL EDUCATIVO E ESPORTIVO
4,30.14.123,REDE ESPORTE FUTEBOL DE CAMPO,3,14 - MATERIAL EDUCATIVO E ESPORTIVO
...,...,...,...,...
626,30.42.972,LUZ EMERGÊNCIA BOTOEIRA INDUSTRIAL DE COMANDO ...,39,42 - FERRAMENTAS
627,30.42.973,LUZ EMERGÊNCIA BOTOEIRA INDUSTRIAL DE COMANDO ...,40,42 - FERRAMENTAS
628,30.44.67,"Fita adesiva, material propileno, 20mm x 30m, ...",11,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS
629,30.44.69,PLACA DE SINALIZAÇÃO DE AMBIENTE,2,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS


,codigo,descricao_lev,qtd_lev,servidor
0,30.24.2578,PREGO com cabeça de 1 1/2 x 13,4,joao.cruz@afogados.ifpe.edu.br
1,30.14.121,REDE ESPORTE BASQUETE,7,elaine.leite@afogados.ifpe.edu.br
2,30.14.171,RAQUETE BADMINTON,36,elaine.leite@afogados.ifpe.edu.br
3,30.14.173,REDE ESPORTE BADMINTON.,6,elaine.leite@afogados.ifpe.edu.br
4,30.14.209,SACOLA/BOLSA PARA TRANSPORTE DE MATERIAL ESPOR...,1,elaine.leite@afogados.ifpe.edu.br
...,...,...,...,...
636,30.21.993,GALHETEIRO DE AÇO INOX.,1,anderson.lima@afogados.ifpe.edu.br
637,30.21.997,"PANELA DE PRESSÃO 4,5L",3,anderson.lima@afogados.ifpe.edu.br
638,30.23.757,"TOGA, BECA PARA AUTORIDADES, EM CETIM NA COR P...",10,newton.souza@afogados.ifpe.edu.br
639,30.28.228,Óculos de proteção confeccionado em uma única ...,0,newton.souza@afogados.ifpe.edu.br


In [5]:
consolidacao = pd.merge(suap_cross, levantamento_cross, on='codigo', how='outer', indicator=True)
consolidacao.to_excel(f'consolidacao/{n_campi}_consolidacao.xlsx', index=False)


consolidacao['diferenca'] = consolidacao['qtd_suap'] - consolidacao['qtd_lev']
display(consolidacao)

pedidos = consolidacao[consolidacao['_merge'].str.contains('both')]
pedidos = pedidos[['codigo', 'descricao_suap', 'qtd_suap', 'qtd_lev', 'diferenca']]

acao = []
for valor in pedidos['diferenca']:
    
    if valor > 0:
        baixa = "pedido no suap"
        acao.append(baixa)
    elif valor < 0:
        baixa = "saida sem pedido"
        acao.append(baixa)
    else:
        baixa = 'adequado'
        acao.append(baixa)

pedidos['acao'] = acao
pedidos.to_excel(f'consolidacao/{n_campi}_pedidos.xlsx', index=False)
display(pedidos)

,codigo,descricao_suap,qtd_suap,ed,descricao_lev,qtd_lev,servidor,_merge,diferenca
0,30.14.117,"APITO, MATERIAL PLÁSTICO, APLICAÇÃO ESPORTE, T...",16,14 - MATERIAL EDUCATIVO E ESPORTIVO,"APITO, MATERIAL PLÁSTICO, APLICAÇÃO ESPORTE, T...",14,elaine.leite@afogados.ifpe.edu.br,both,2
1,30.14.120,BOLA TENIS DE MESA,17,14 - MATERIAL EDUCATIVO E ESPORTIVO,BOLA TENIS DE MESA,12,elaine.leite@afogados.ifpe.edu.br,both,5
2,30.14.121,REDE ESPORTE BASQUETE,8,14 - MATERIAL EDUCATIVO E ESPORTIVO,REDE ESPORTE BASQUETE,7,elaine.leite@afogados.ifpe.edu.br,both,1
3,30.14.122,REDE ESPORTE FUTSAL,10,14 - MATERIAL EDUCATIVO E ESPORTIVO,REDE ESPORTE FUTSAL,10,elaine.leite@afogados.ifpe.edu.br,both,0
4,30.14.123,REDE ESPORTE FUTEBOL DE CAMPO,3,14 - MATERIAL EDUCATIVO E ESPORTIVO,REDE ESPORTE FUTEBOL DE CAMPO,3,elaine.leite@afogados.ifpe.edu.br,both,0
...,...,...,...,...,...,...,...,...,...
631,30.42.972,LUZ EMERGÊNCIA BOTOEIRA INDUSTRIAL DE COMANDO ...,39,42 - FERRAMENTAS,LUZ EMERGÊNCIA BOTOEIRA INDUSTRIAL DE COMANDO ...,39,dario.frazao@afogados.ifpe.edu.br,both,0
632,30.42.973,LUZ EMERGÊNCIA BOTOEIRA INDUSTRIAL DE COMANDO ...,40,42 - FERRAMENTAS,LUZ EMERGÊNCIA BOTOEIRA INDUSTRIAL DE COMANDO ...,40,dario.frazao@afogados.ifpe.edu.br,both,0
633,30.44.67,"Fita adesiva, material propileno, 20mm x 30m, ...",11,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,"Fita adesiva, material propileno, 20mm x 30m, ...",10,dario.frazao@afogados.ifpe.edu.br,both,1
634,30.44.69,PLACA DE SINALIZAÇÃO DE AMBIENTE,2,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,PLACA DE SINALIZAÇÃO DE AMBIENTE,2,dario.frazao@afogados.ifpe.edu.br,both,0


,codigo,descricao_suap,qtd_suap,qtd_lev,diferenca,acao
0,30.14.117,"APITO, MATERIAL PLÁSTICO, APLICAÇÃO ESPORTE, T...",16,14,2,pedido no suap
1,30.14.120,BOLA TENIS DE MESA,17,12,5,pedido no suap
2,30.14.121,REDE ESPORTE BASQUETE,8,7,1,pedido no suap
3,30.14.122,REDE ESPORTE FUTSAL,10,10,0,adequado
4,30.14.123,REDE ESPORTE FUTEBOL DE CAMPO,3,3,0,adequado
...,...,...,...,...,...,...
631,30.42.972,LUZ EMERGÊNCIA BOTOEIRA INDUSTRIAL DE COMANDO ...,39,39,0,adequado
632,30.42.973,LUZ EMERGÊNCIA BOTOEIRA INDUSTRIAL DE COMANDO ...,40,40,0,adequado
633,30.44.67,"Fita adesiva, material propileno, 20mm x 30m, ...",11,10,1,pedido no suap
634,30.44.69,PLACA DE SINALIZAÇÃO DE AMBIENTE,2,2,0,adequado
